In [3]:
import time
# fire 
# news
# NLP -> 

import numpy as np
def relu(x, deriv = False):
    #ReLu logistic compaction
    if(x > 0):
        if(deriv): return 1
        else: return x
    else:
        return 0
def sigmoid(x, deriv = False):
    # Sigmoid logistic compaction
    if (deriv):
        sig = sigmoid(x, False)
        return sig * (1 - sig)
    else:
        return 1 / (1 + np.exp(-x))
def tanh(x, deriv = False):
    # Tanh logistic compaction
    if(deriv):
        return 1 - np.power(tanh(x, False), 2)
    else:
        return (1 - np.exp(-2*x))/(1 + np.exp(-2*x))

def forward_propogate(X, W1, b1, W2, b2):
    #Forward Propogation
    Z1 = np.dot(W1.T, X) + b1
    A1 = tanh(Z1, False)
    
    Z2 = np.dot(W2.T, A1) + b2;
    A2 = sigmoid(Z2, False)
    return (A1, A2, Z1, Z2)

def propogate(X, W1, b1, W2, b2):
    return forward_propogate(X, W1, b1, W2, b2)[1];

def error(A, Y):
    # Error function
    return -(Y*np.log(A) + (1 - Y)*np.log(1 -A))

def train(X, Y, N, act, seed, steps, l_rate):
    m = Y.shape[1]   # num training examples
    # N[0] = nx
    # N[1] = numNeurons layer1
    # column is 1 training example, with each row being another neuron for each layer
    
    # Setup Seed
    np.random.seed(seed)
    #N[0] is A0, A1 is N[1]
    # Each column is a neuron, each row is a weight for previous row
    W1 = np.random.randn(N[0], N[1])
    W2 = np.random.randn(N[1],N[2])
    
    # Not necessary for b value to init non zero
    b1 = np.random.randn(N[1],1) 
    b2 = np.random.randn(N[2],1)
    
    for i in range(steps):
        # Determine actual values, and find error compared to the predicted values
        (A1, A2, Z1, Z2) = forward_propogate(X, W1, b1, W2, b2)
        #print(A1.shape) # double check
        J = np.average(error(A2, Y)) #mean error
        
        # Start Backpropogation
        # Compute Derivatives
        #dZ = A - Y
        dZ2 = A2 - Y
        dW2 = 1/m * np.dot(dZ2, A1.T).T  # dividing by m kinda unnecessary # N[1]xN[2] dim
        dB2 = 1/m * np.sum(dZ2, axis = 1, keepdims = True)
        
        #print(dZ2)
        #print(np.dot(W2.T, dZ2)) #THE BUGGER
        #print(np.dot(W2, dZ2)) #FIXED
        
        dZ1 = np.dot(W2, dZ2) * tanh(Z1, True) # elementwise multiplication of dot and derivative
        
        dW1 = 1/m * np.dot(dZ1, X.T).T  # dividing by m kinda unnecessary
        dB1 = 1/m * np.sum(dZ1, axis = 1, keepdims = True)
        
        # Step the variables according to the slopes
        #print(W2)
        #print(dW2)
        W2 = W2 - l_rate * dW2
        b2 = b2 - l_rate * dB2
        
        W1 = W1 - l_rate * dW1
        b1 = b1 - l_rate * dB1
        
    return (W1, b1, W2, b2, J)

N = np.array([3, 6, 1]) #size of NN, 1st is input layer, 2nd is the hidden layer, 3rd is the output layer 
act = 2 # temp,in real this would store the activation functions, in this example its hard coded
iterations = 500
learning_rate = 5 #step size scalar
seed = 0 #Deterministic Seed 
# Training examples

# 2x2 adder
X = np.array([ # First 2 plus second 2
    [0, 1, 0, 1],
    [0, 0, 1, 1],
    [0, 0, 1, 0]
]) 

# Predicted Values
Y = np.array([ # 1st out 2nd out, carry out
    [0, 1, 1, 0]
    
]) 

# Testing data
test = np.array([
    [0, 1, 0, 1],
    [0, 0, 1, 1],
    [1, 1, 1, 1]
])

tic = time.time() #Start timer
# Train to find weights, and b value
print("Training...")
(W1, b1, W2, b2, J) = train(X, Y, N, act, seed, iterations, learning_rate)

toc = time.time()
np.set_printoptions(suppress=True) # Ensure data is in decimal format
print("Calculating Test Values...")

# Print values
print("Weight 1: ", end = "")
print(W1)
print("B1 value: ", end = "")
print(b1)

print("Weight 2: ", end = "")
print(W2)
print("B2 value: ", end = "")
print(b2)
print()

print("Error: " + str(J * 100) + "%")
print("Output: ", end = "")
output = propogate(test, W1, b1, W2, b2) # use testing data

print(output) # Use testing data
print(np.round(output, 3))
print("\nDuration " + str(1000 * (toc - tic)) + "ms")


Training...
Calculating Test Values...
Weight 1: [[ 1.74876917  0.69037531  2.39750037  4.22074372  2.84367052 -3.2793029 ]
 [ 0.94607663  2.95984354 -1.16858068  1.84487069 -4.98383001  1.70450212]
 [ 0.75977931 -1.4332936  -2.70919338  1.83731361  1.59713677  0.32270697]]
B1 value: [[ 2.03825438]
 [-2.35786053]
 [ 1.84455744]
 [-1.58342913]
 [-1.04757252]
 [-0.70771107]]
Weight 2: [[ 0.4064997 ]
 [-3.83561108]
 [-4.12922378]
 [ 4.77561435]
 [ 4.31632089]
 [ 1.32563674]]
B2 value: [[0.13963662]]

Error: 0.03709094399703563%
Output: [[0.9999597  0.99976644 0.99981893 0.30398662]]
[[1.    1.    1.    0.304]]

Duration 144.15526390075684ms


In [20]:


# Neural Network to solve Rubiks Cube!!!!!!!!!!

# What is Prolog


# IMPLEMENT 15 puzzle slide, when you learn Recurrent Neuaral Networks
# Initial State: [1,6,3,8,10,2,7,15,13,5,4,0,9,14,11,12]
# Goal State   : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]
# 0 is empty
# Operations: UP DOWN LEFT RIGHT if: vacant spot adjacent
# Legal Moves Calculator: (Assuming 1D Array)
#     LEFT: IF (pos(0) % 4 != 3), the tile right to it
#    RIGHT: IF (pos(0) % 4 != 0), the tile left to it
#     DOWN: IF (pos(0) > 3), the tile 4 to the left of it
#       UP: IF (pos(0) < 12), the tile 4 to the right of it

# Manhattan Distance (RAW STEPS AWAY FOR EACH TILE)